In [ ]:
# ==============================================================================
# 2. AUTORYZACJA EARTH ENGINE (POPRAWIONA)
# ==============================================================================
import ee

# 1. Usuwamy stare poświadczenia (jeśli były błędne)
!rm -rf /root/.config/earthengine

print("Rozpoczynam autoryzację...")

# 2. Wymuszamy tryb 'notebook' - to pokaże link i okienko na kod
# Usunąłem parametr project='...', system sam przypisze Twój domyślny.
try:
    ee.Initialize()
    print("GEE zainicjowane pomyślnie (znaleziono zapisane dane).")
except Exception:
    print("Wymagana autoryzacja. Kliknij w link, autoryzuj i wklej kod poniżej.")
    # force=True wymusza nowe logowanie
    ee.Authenticate(auth_mode='notebook', force=True) 
    ee.Initialize()
    print("GEE zainicjowane po autoryzacji.")

In [ ]:
# ==============================================================================
# 1. INSTALACJA
# ==============================================================================
!pip install geemap rasterio onnxruntime-gpu geopandas shapely -q

import ee
import geemap
import os
import rasterio
import onnxruntime as ort
import numpy as np
import geopandas as gpd
from shapely.geometry import shape
from rasterio.features import shapes
import shutil
import glob
from zipfile import ZipFile
import sys
import time

# ==============================================================================
# 2. AUTORYZACJA
# ==============================================================================
print("1. Autoryzacja Earth Engine...")
try:
    ee.Initialize()
    print("GEE zainicjowane.")
except Exception:
    print("Wymagana autoryzacja. Kliknij link, autoryzuj i wklej kod.")
    ee.Authenticate(auth_mode='notebook', force=True)
    ee.Initialize()
    print("GEE zainicjowane po autoryzacji.")

# ==============================================================================
# 3. KONFIGURACJA
# ==============================================================================
CONFIG = {
    # --- ŚCIEŻKI ---
    "MODEL_PATH": "/kaggle/input/pv-model/PV_Segmentation_UNETPP.onnx",
    "OUTPUT_DIR": "/kaggle/working/wyniki_pv",
    
    # --- OBSZAR I SIATKA ---
    "ROI_TYPE": "Country", 
    "REGION_NAME": "Poland", 
    "GRID_SIZE_DEG": 0.25, 
    "EXPORT_CRS": "EPSG:2180", 
    
    # --- OPTYMALIZACJA ---
    "MAX_NODATA_RATIO": 0.40,  # Pomiń jeśli > 40% to czerń
    "TILE_OVERLAP": 0.15,      # 15% zakładki
    
    # --- DATY ---
    "START_DATE": "2024-04-01",
    "END_DATE": "2024-09-30",
    "MAX_CLOUD_PCT": 10, 
    
    # --- WIZUALIZACJA ---
    "VIS_PARAMS": {'bands': ['B12', 'B11', 'B8A'], 'min': 0, 'max': 3000, 'gamma': 1.2},
    
    # --- ZAPIS ---
    "SAVE_VECTORS": True,
    "SAVE_RASTER": False, 
    "MIN_FARM_AREA_M2": 2000, 
    "THRESHOLD": 0.5          
}

# ==============================================================================
# 4. FUNKCJE POMOCNICZE
# ==============================================================================

def get_roi():
    if CONFIG['ROI_TYPE'] == "Country":
        print(f"Pobieram granice: {CONFIG['REGION_NAME']}...")
        return ee.FeatureCollection("FAO/GAUL/2015/level0") \
                 .filter(ee.Filter.eq('ADM0_NAME', CONFIG['REGION_NAME'])).geometry()
    else:
        return ee.Geometry.Rectangle([14.4, 52.0, 15.0, 52.3])

def get_processed_image(roi):
    collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
                  .filterDate(CONFIG['START_DATE'], CONFIG['END_DATE'])
                  .filterBounds(roi)
                  .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', CONFIG['MAX_CLOUD_PCT'])))
    
    if collection.size().getInfo() == 0: return None
    image = collection.median().clip(roi)
    return image.visualize(**CONFIG['VIS_PARAMS'])

def check_is_empty(tif_path, threshold_ratio):
    with rasterio.open(tif_path) as src:
        h, w = src.height, src.width
        data = src.read(1, out_shape=(1, int(h/10), int(w/10))) # Szybki podgląd
        zeros = np.count_nonzero(data == 0)
        ratio = zeros / data.size
        return ratio > threshold_ratio, ratio

def run_detection_with_overlap(tif_path, session):
    with rasterio.open(tif_path) as src:
        meta = src.meta.copy()
        img = src.read()
        if img.shape[0] > 3: img = img[:3, :, :]
        
        c, h, w = img.shape
        tile_size = 1024
        overlap_ratio = CONFIG['TILE_OVERLAP']
        stride = int(tile_size * (1 - overlap_ratio))
        
        pad_h = tile_size + stride
        pad_w = tile_size + stride
        img_padded = np.pad(img, ((0,0), (0, pad_h), (0, pad_w)), mode='constant')
        ph, pw = img_padded.shape[1], img_padded.shape[2]
        
        result_mask = np.zeros((ph, pw), dtype=np.float32)
        input_name = session.get_inputs()[0].name
        
        for y in range(0, h, stride):
            for x in range(0, w, stride):
                tile = img_padded[:, y:y+tile_size, x:x+tile_size]
                if tile.shape[1] != tile_size or tile.shape[2] != tile_size: continue

                inp = tile[np.newaxis, :, :, :].astype(np.float32)
                res = session.run(None, {input_name: inp})
                pred_tile = res[0][0, 0, :, :]
                
                # Nakładanie wyników (MAX)
                current_area = result_mask[y:y+tile_size, x:x+tile_size]
                result_mask[y:y+tile_size, x:x+tile_size] = np.maximum(current_area, pred_tile)
                
        final_mask = result_mask[:h, :w]
        binary = (final_mask > CONFIG['THRESHOLD']).astype(rasterio.uint8)
        return binary, meta

def save_geojson(binary_mask, meta, output_path):
    if binary_mask.max() == 0: return False
    shapes_gen = shapes(binary_mask, mask=(binary_mask > 0), transform=meta['transform'])
    geoms = [shape(s) for s, v in shapes_gen]
    if not geoms: return False
    
    gdf = gpd.GeoDataFrame({'geometry': geoms}, crs=meta['crs'])
    gdf['area_m2'] = gdf.geometry.area
    gdf_filtered = gdf[gdf['area_m2'] > CONFIG['MIN_FARM_AREA_M2']]
    
    if len(gdf_filtered) > 0:
        gdf_filtered.to_file(output_path, driver='GeoJSON')
        return True
    return False

# ==============================================================================
# 5. START (GŁÓWNA PĘTLA)
# ==============================================================================

def main():
    if not os.path.exists(CONFIG['OUTPUT_DIR']):
        os.makedirs(CONFIG['OUTPUT_DIR'], exist_ok=True)
        
    if not os.path.exists(CONFIG['MODEL_PATH']):
        print("BŁĄD: Brak modelu! Sprawdź ścieżkę w CONFIG.")
        return

    try:
        session = ort.InferenceSession(CONFIG['MODEL_PATH'], providers=['CUDAExecutionProvider', 'CPUExecutionProvider'])
        print("Model na GPU.")
    except:
        session = ort.InferenceSession(CONFIG['MODEL_PATH'], providers=['CPUExecutionProvider'])
        print("Model na CPU.")
    
    roi = get_roi()
    print(f"Siatka {CONFIG['GRID_SIZE_DEG']} stopni. Generuję listę...")
    grid = geemap.fishnet(roi, h_interval=CONFIG['GRID_SIZE_DEG'], v_interval=CONFIG['GRID_SIZE_DEG'])
    grid_list = grid.toList(grid.size()).getInfo()
    total = len(grid_list)
    print(f"-> Kafelków do zrobienia: {total}")
    
    total_found = 0
    skipped_nodata = 0
    
    for i, item in enumerate(grid_list):
        tile_id = f"tile_{i:04d}"
        json_out = os.path.join(CONFIG['OUTPUT_DIR'], f"{tile_id}.geojson")
        temp_tif = f"/kaggle/working/temp_{tile_id}.tif"
        
        t_start_all = time.time()

        if os.path.exists(json_out): 
            print(f"{tile_id} już istnieje. Pomijam.")
            continue

        try:
            # 1. POBIERANIE
            t_dl_start = time.time()
            tile_geom = ee.Geometry(item['geometry'])
            img_ee = get_processed_image(tile_geom)
            
            if img_ee is None: continue
            
            geemap.ee_export_image(
                img_ee, filename=temp_tif, scale=10, 
                region=tile_geom, crs=CONFIG['EXPORT_CRS'], file_per_band=False
            )
            t_dl_end = time.time()
            dl_time = t_dl_end - t_dl_start
            
            if not os.path.exists(temp_tif): continue
            
            # 2. PRZETWARZANIE
            t_proc_start = time.time()
            
            # Check NoData
            is_empty, ratio = check_is_empty(temp_tif, CONFIG['MAX_NODATA_RATIO'])
            status_msg = ""
            
            if is_empty:
                skipped_nodata += 1
                status_msg = f"Skip (Pusty: {ratio:.0%})"
                os.remove(temp_tif)
            else:
                mask, meta = run_detection_with_overlap(temp_tif, session)
                if CONFIG['SAVE_VECTORS']:
                    saved = save_geojson(mask, meta, json_out)
                    if saved:
                        total_found += 1
                        status_msg = f"WYKRYTO FARMĘ!"
                    else:
                        status_msg = "Brak obiektów."
                os.remove(temp_tif)
            
            t_proc_end = time.time()
            proc_time = t_proc_end - t_proc_start
            
            # Print
            print(f"[{i+1}/{total}] {tile_id} | DL: {dl_time:.1f}s | AI: {proc_time:.1f}s | {status_msg}")
                    
        except Exception as e:
            print(f"\nError {tile_id}: {e}")
            if os.path.exists(temp_tif): os.remove(temp_tif)

    print(f"\n\n🏁 Koniec! Wykryto w: {total_found}. Pominięto pustych: {skipped_nodata}.")
    
    print("Pakowanie ZIP...")
    shutil.make_archive("/kaggle/working/Wyniki_Farmy", 'zip', CONFIG['OUTPUT_DIR'])
    print("GOTOWE! Pobierz ZIP.")

if __name__ == "__main__":
    main()

In [ ]:
import geopandas as gpd
import pandas as pd
import glob
import os

# Folder z wynikami (z konfiguracji głównego skryptu)
OUTPUT_DIR = "/kaggle/working/wyniki_pv"
FINAL_FILE = "/kaggle/working/Farmy_PV_Polska_CALOSC.geojson"

print("Szukam plików z farmami...")
all_files = glob.glob(os.path.join(OUTPUT_DIR, "*.geojson"))

if not all_files:
    print("Nie znaleziono żadnych plików .geojson! (Może nic nie wykryto lub zły folder?)")
else:
    print(f"Znaleziono {len(all_files)} kafelków z wykrytymi farmami. Scalam...")
    
    gdf_list = []
    for file in all_files:
        try:
            gdf = gpd.read_file(file)
            # Dodajemy ID kafelka do tabeli (żebyś wiedział skąd to jest)
            gdf['tile_id'] = os.path.basename(file).replace(".geojson", "")
            gdf_list.append(gdf)
        except Exception as e:
            print(f"Błąd pliku {file}: {e}")

    if gdf_list:
        merged_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True), crs=gdf_list[0].crs)
        
        # Opcjonalne czyszczenie (np. usunięcie duplikatów na stykach kafelków)
        # merged_gdf = merged_gdf.dissolve() 
        
        print(f"Zapisuję: {FINAL_FILE}")
        print(f"Łącznie wykryto: {len(merged_gdf)} obiektów.")
        
        merged_gdf.to_file(FINAL_FILE, driver='GeoJSON')
        print("GOTOWE! Pobierz ten plik z panelu Output.")
    else:
        print("!!!Pusta lista!!!")